In [ ]:
# This notebook assumes to be running from your FireCARES VM (eg. python manage.py shell_plus --notebook --no-browser)

import sys
import os
import time
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.realpath('..'))
import folium
import django
django.setup()
from django.db import connections
from pretty import pprint
from firecares.firestation.models import FireDepartment, FireStation, NFIRSStatistic
from django.db.models import Avg, Max, Min, Q
from django.contrib.gis.geos import GEOSGeometry
from IPython.display import display
from firecares.utils import lenient_summation, dictfetchall
pd.set_option("display.max_rows",100)

def display_geom(geom):
    _map = folium.Map(location=[geom.centroid.y, geom.centroid.x],
                      tiles='Stamen Toner')
    _map.choropleth(geo_str=geom.geojson, line_weight=0, fill_opacity=0.2, fill_color='green')
    ll = geom.extent[1::-1]
    ur = geom.extent[3:1:-1]
    _map.fit_bounds([ll, ur])

    return _map

In [ ]:
q = """select fd.id as firecares_id, name, address_line1 as hq_addr_line1, address_line2 as hq_addr_line2, city as hq_city, state_province as hq_state, postal_code as hq_postal_code, ST_X(ad.geom) as hq_x, ST_Y(ad.geom) as hq_y
from firestation_firedepartment fd
inner join firecares_core_address ad
    on fd.headquarters_address_id = ad.id
where not fd.archived;"""

df = pd.read_sql_query(q, connections['default'])

In [ ]:
df.to_csv('/tmp/fd_geocodes.csv')